In [13]:
all([True, True, False])

False

In [9]:
from typing import Dict

def is_different_from_default(result: Dict[str, str]) -> bool:
    default_values = {
        'date_range': 'N/A',
        'from_date': 'N/A', 
        'to_date': 'N/A',
        'product': 'All',
        'product_detail': 'All',
        'level': 'All',
        'user': 'N/A'
    }
    
    for key, default_value in default_values.items():
        if key in result and result[key] != default_value:
            return True
            
    return False

In [10]:
result = {'date_range': 'N/A', 'from_date': 'N/A', 'to_date': 'N/A', 'product': 'Sportsbook', 'product_detail': 'All', 'level': 'All', 'user': 'N/A'}
is_different_from_default(result)


True

In [11]:
import pandas as pd

PRODUCT = [
    "Sportsbook", "Number Game", "Virtual Sports", "Saba Casino", "RNG Keno", "AG Casino",
    "Sportsbook 2", "RNG Slot", "Cricket", "Allbet", "Macau Games", "Cash Out", "Lottery",
    "Voidbrige Jackpot", "Exchange", "PP", "Arcadia Gaming", "SG", "Saba Promotion",
    "Virtual Games", "SA Gaming", "Table Game", "Live Casino", "SABA.games", "Togel 4D",
    "RNG Games", "AE Sexy", "IBCBet Live Casino", "BBIN", "GPI", "WM", "ION", "RNG Casino",
    "Saba Virtual Sports", "PG Soft", "Joker", "BG", "MaxGame", "Habanero", "CG",
    "Sports Lottery", "PP Live Casino", "Vgaming", "AdvantPlay", "AdvantPlay Mini", 
    "Bitcoin", "FGG", "SEAL Entertainment", "MG-RNG", "Player Tips", "Jili", "YeeBet", 
    "UU SLOTS", "Live22", "WE Live Casino", "Saba Coins", "Yolo Play", "Nextspin",
    "SABA xD", "FastSpin", "FA CHAI", "GPI Live Casino", "PLAYSTAR", "HOTDOG", "ON Casino",
    "Smartsoft", "Playtech", "SABA Keno", "ASKMESLOT", "Funky Games"
]

PRODUCT_DETAIL = [
    "SABA Basketball", "SABA Basketball PinGoal", "SABA E-Sports PinGoal",
    "SABA Other Sports", "SABA Soccer", "SABA Soccer PinGoal", "SABA Tennis",
    "Sportsbook", "Allbet Promotion", "PP Jackpot Contribution", "PP Jackpot Prize",
    "SG Jackpot Contribution", "SG Jackpot Prize", "AE Sexy Lucky Draw",
    "Vgaming Promotion Prize", "MG-RNG Promotion Prize", "Live22 Promotion Prize",
    "FastSpin Promotion Prize", "FA CHAI Promotion Prize", "ASKMESLOT Promotion Prize"
]

LEVEL = ['Super Agent', 'Master Agent', 'Agent', 'Direct Member']

# customer = pd.read_csv(r'/home/hoangtv/Desktop/Long/llm-tuning/data/customer.csv')
customer = pd.read_csv(r'D:\\Desktop\\AlphaReportChatbot\\data\\customer.csv')
USER = customer.dropna()['username'].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\\\Desktop\\\\AlphaReportChatbot\\\\data\\\\customer.csv'

In [2]:
import faiss
import requests
import json
from typing import Dict, Any, Optional
import tqdm

def llm(
    user_prompt: str,
    system_prompt: str,
    format_schema: Dict[str, Any] = None,
    api: str = 'https://ollama.selab.edu.vn',
    endpoint: str = '/api/chat',
    model: str = "qwen2.5:14b"
) -> Dict[str, str]:
    
    headers = {
      "Content-Type": "application/json"
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]


    if endpoint == '/api/chat' and format_schema is not None:
        data = {
            "model": model,
            "messages": messages,
            "format": format_schema,
            "stream": False
        }
    
    elif endpoint == '/api/generate':
        data = {
            "model": model,
            "prompt": messages,
            "option": {
              "temperature": 0.4  
            },
            "stream": False
        }

    url = f'{api}{endpoint}'
    
    response = requests.post(url, headers=headers, json=data, timeout=600)


    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")

    response_data = response.json()
    
    return json.loads(response_data)

# ***Prepare function to call***

In [20]:
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import torch

def faiss_indexing(data_embeddings, path_save):
    # indexing
    cpu_index = faiss.IndexFlatIP(768)
    for embedding in tqdm.tqdm(data_embeddings, colour='green', desc='Indexing'):
        embedding = embedding.astype(np.float32).reshape(1, -1)
        cpu_index.add(embedding)

    # Save vector database
    faiss.write_index(cpu_index, path_save)
    
     
def save_pickle(
    data, 
    file_path='bm25_index.pkl'
) -> None:
    # Save tokenized corpus into a pickle file
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)


def bm25_indexing(contents, path_save):
    # indexing
    tokenized_corpus = []
    for doc in tqdm.tqdm(contents, colour='green'):
        doc_tokens = doc.split()
        tokenized_corpus.append(doc_tokens)

    # Save into pickle file
    save_pickle(tokenized_corpus, path_save)


def embedding_data(
    model_embedding,
    contents
) -> torch.Tensor:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model_embedding.to(device)
    embeddings = model.encode(contents)
    
    return embeddings

In [5]:
function_metadata = {
    "/get_winlost_report": [
        "As an Agent, I want to retrieve Win/Loss reports via the chatbot so that I can track user performance and betting trends.",
        "The chatbot should allow filtering by date, product, and user level. It should present results in a structured format (text/table).",
        "The chatbot should support exporting reports (CSV/PDF).",
        "The chatbot should allow agents to compare win/loss trends across different time periods to detect anomalies or high-risk users.",
        "The report should highlight the biggest winners and losers in a given period.",
        "The chatbot should notify agents of any unusual betting patterns that might require further investigation."
    ],
    "/get_outstanding_report": [
        "As an Agent, I want to access outstanding reports via the chatbot so that I can monitor unsettled transactions.",
        "The chatbot should categorize outstanding reports based on transaction type (e.g., deposits, withdrawals, bets).",
        "The chatbot should provide real-time updates on pending settlements.",
        "The chatbot should allow filtering by user, date, and transaction amount to prioritize pending payments.",
        "The chatbot should alert agents if outstanding transactions exceed a defined threshold."
    ],
    "/get_statement_report": [
        "As an Agent, I want to generate a statement report via the chatbot so that I can track transactions and financial summaries.",
        "The chatbot should retrieve statements for a specific date range.",
        "The chatbot should allow exporting statement reports in multiple formats (CSV, PDF).",
        "The chatbot should summarize total deposits, withdrawals, and net balance for each statement period.",
        "The chatbot should allow agents to generate reports for individual users or aggregated across all users.",
        "The chatbot should provide a breakdown of different transaction categories (e.g., bet winnings, bonuses, refunds)."
    ],
    "/create_customer_information": [
        "As an Agent, I want to create a new member via the chatbot so that I can quickly onboard new users.",
        "The chatbot should collect required user information (name, contact, account type).",
        "The chatbot should validate input data before submission to prevent duplicate or incorrect registrations.",
        "The chatbot should confirm successful account creation and generate a welcome message.",
        "The chatbot should assign default account limits based on user type.",
        "The chatbot should support bulk user registration via file upload or batch processing."
    ],
    "/get_member_details": [
        "As an Agent, I want to retrieve member details via the chatbot so that I can verify user accounts.",
        "The chatbot should allow searching by username or user ID.",
        "The chatbot should display user level, account settings, and account status.",
        "The chatbot should highlight any account restrictions or previous risk flags.",
        "The chatbot should provide an account history summary, including past transactions and betting activities.",
        "The chatbot should support multi-level filtering (e.g., by region, VIP status, risk category)."
    ],
    "/get_problem_accounts": [
        "As an Agent, I want to view a list of problem accounts via the chatbot so that I can take corrective actions.",
        "The chatbot should display problem accounts with their current status and reason for being flagged.",
        "The chatbot should allow filtering by issue type (e.g., fraud suspicion, payment issues, excessive losses).",
        "The chatbot should generate alerts for accounts that repeatedly trigger risk warnings.",
        "The chatbot should provide recommendations on how to handle each problem account."
    ],
    "/get_bet_tickets": [
        "As an Agent, I want to search for specific bet tickets via the chatbot so that I can track individual bet details.",
        "The chatbot should allow searching by ticket ID, user, and date.",
        "The chatbot should display bet details, including bet type, odds, and payout status.",
        "The chatbot should provide real-time updates on active bets.",
        "The chatbot should allow agents to filter by bet outcome (e.g., won, lost, pending)."
    ],
    "/get_running_bets": [
        "As an Agent, I want to retrieve running bets via the chatbot so that I can verify member betting behavior and forecast risk amount.",
        "The chatbot should allow filtering by date, product, and user.",
        "The chatbot should display bet details and status (e.g., pending, live, settled).",
        "The chatbot should allow exporting bet reports for further analysis.",
        "The chatbot should highlight bets with unusually high stakes."
    ],
    "/get_cancelled_bets": [
        "As an Agent, I want to view cancelled bets via the chatbot so that I can investigate betting issues.",
        "The chatbot should display cancelled bets with reasons (e.g., voided, system error, user cancellation).",
        "The chatbot should allow filtering by date and user.",
        "The chatbot should identify patterns in cancelled bets (e.g., frequent cancellations by the same user).",
        "The chatbot should generate reports on the impact of cancelled bets on total revenue."
    ],
    "/get_bet_forecasting": [
        "As an Agent, I want the chatbot to generate total bets so that I can mitigate potential losses.",
        "The chatbot should summarize major bet types, the number of tickets, and stakes on running tickets of each event.",
        "The chatbot should predict win/loss amounts based on current scores.",
        "The chatbot should display the Total Forecast Win/Loss of each Live event.",
        "The chatbot should alert agents when projected losses exceed a predefined threshold."
    ],
    "/get_winlost_limit": [
        "As an Admin, I want to get a member’s win/loss limit via the chatbot so that I can assess their risk level.",
        "The chatbot should retrieve win/loss limit settings for specific members.",
        "The chatbot should notify admins if a user approaches or exceeds their win/loss limit.",
        "The chatbot should provide historical data on win/loss limit breaches.",
        "The chatbot should allow admins to modify limits based on risk assessment."
    ],
    "/set_risk_thresholds": [
        "As an Agent, I want to set risk thresholds via the chatbot so that I can automatically flag risky bets.",
        "The chatbot should allow setting limits for winning/losing thresholds.",
        "The chatbot should trigger alerts when thresholds are exceeded.",
        "The chatbot should log all risk threshold violations.",
        "The chatbot should allow different threshold levels based on user category (e.g., high-risk users, VIP users)."
    ],
    "/analyze_betting_behavior": [
        "As an Agent, I want to analyze user betting behavior via the chatbot so that I can identify potential problem gamblers or fraud cases.",
        "The chatbot should generate a user profile based on betting patterns, including preferred sports, bet frequency, and risk levels.",
        "The chatbot should detect unusual betting behaviors, such as excessive stakes or repeated last-minute bets.",
        "The chatbot should segment users into different categories based on their betting style (e.g., casual, high-risk, professional).",
        "The chatbot should provide trend analysis, showing how a user’s betting habits change over time."
    ]
}

In [17]:
# Download from the 🤗 Hub
model_embedding = SentenceTransformer("hiieu/halong_embedding")

function_description = list(function_metadata.values())

def flatten_list_2d(list_2d):
    return [item for sublist in list_2d for item in sublist]

function_description = flatten_list_2d(function_description)

# Generate embeddings for the flattened descriptions
embeddings = embedding_data(
    model_embedding=model_embedding,
    contents=function_description
)

In [30]:
function_description.index(function_metadata['/get_outstanding_report'][2])

8

In [19]:
faiss_indexing(embeddings, 'function_description.index')

Indexing: 100%|██████████| 69/69 [00:00<?, ?it/s]


In [22]:
bm25_indexing(function_description, 'funtion_description.pkl')

100%|██████████| 69/69 [00:00<00:00, 199178.92it/s]


# ***Function calling***

In [23]:
def extract_keywords(query):
    
    system_prompt = """
    You are an AI assistant trained to extract keywords from user queries. Your task is to identify the most relevant and meaningful keywords from the given sentence. Focus on nouns, proper nouns, and key phrases that convey the core meaning of the query. Avoid extracting common stop words or irrelevant terms.

    # **Key Guidelines:**
    - Extract only the most relevant keywords.
    - Focus on nouns, proper nouns, and key phrases.
    - Avoid stop words or irrelevant terms.
    - Ensure the extracted keywords are concise and meaningful.

    # ***Example Scenarios:***
    - ***Input***: "Get me a Win Loss Detail Report for Direct Member who played Sportsbook."
    - ***Output***: ["Win Loss Detail Report", "Direct Member", "Sportsbook"]

    - ***Input***: "Show me the betting trends for last month."
    - ***Output***: ["betting trends", "last month"]

    - ***Input***: "Retrieve member details for user123."
    - ***Output***: ["member details", "user123"]
    """
    
    format_schema = {
        "type": "object",
        "properties": {
            "keywords": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": ["keywords"]
    }
    
    user_prompt = f'''
        Please extract the keywords from this sentences: {query}
    '''

    keywords = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=format_schema
    )

    return keywords['message']['content']

In [31]:
def load_bin(path):
    cpu_index = faiss.read_index(path) 
    return cpu_index

In [32]:
import pickle

def load_pickle(file_path: str):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
from rank_bm25 import BM25Okapi


def bm25_keyword_search(query, corpus, topk):

  bm25 = BM25Okapi(corpus)
  tokenized_query = extract_keywords(query).keywords
  function_names = bm25.get_top_n(tokenized_query, corpus, n=topk)

  return function_names

def semantic_keyword_search(query, corpus, model_embedding, cpu_index, topk):
  keywords = extract_keywords(query).keywords

  function_names = []
  for keyword in keywords:
    prompt_embedding = model_embedding.encode([keyword])
    prompt_embedding = np.array(prompt_embedding)
    scores, indices = cpu_index.search(prompt_embedding, topk)
    contexts = [', '.join(data[i]['headers']) + ' ' + data[i]['content'] for i in indices.flatten().tolist()]
    function_names.extend(contexts)

  return function_names


def semantic_search(query, corpus, model_embedding, cpu_index, topk):
  prompt_embedding = model_embedding.encode([query])
  prompt_embedding = np.array(prompt_embedding)
  scores, indices = cpu_index.search(prompt_embedding, topk)
  function_names = [', '.join(data[i]['headers']) + ' ' + data[i]['content'] for i in indices.flatten().tolist()]

  return function_names

def hybrid_search(query, topk):
  
  bm25_index = load_pickle('function_description.pkl')
  faiss_index = load_bin('function_description.index')
  
  
  
  bm25_contexts = bm25_keyword_search(query, bm25_index, topk)
  halong_contexts = semantic_keyword_search(query, model_embedding, faiss_index, topk)
  semantic_contexts = semantic_search(query, model_embedding, faiss_index, topk)

  function_names = bm25_contexts + halong_contexts + semantic_contexts
  function_names = list(set(function_names))

  return function_names

In [ ]:

from sentence_transformers import CrossEncoder
import torch.nn as nn

model_reranking = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

sigmoid = nn.Sigmoid()

def hybrid_search_with_reranking(query, topk):
  contexts = hybrid_search(query, topk)
  formated_contexts = [[query, context] for context in contexts]

  scores = model_reranking.predict(formated_contexts, activation_fct=sigmoid)
  top_k_values = np.sort(scores)[-topk:][::-1]

  top_k_indices = np.argsort(scores)[-topk:][::-1]
  print(top_k_values)

  contexts = np.array(contexts)
  best_contexts = contexts[top_k_indices].tolist()

  return best_contexts

In [ ]:
def function_calling(
    user_prompt: str,
    system_prompt: str,
    vector_db: faiss.IndexFlatL2,
    db_contains_function_name: Dict[str] 
) -> str:
    
    pass

# ***Build conversation chatbot***

In [ ]:
def conversation_agent(
    query: str,
    parameters: Optional[Dict[str, str]]
) -> Dict[str, str]:
    
    system_prompt = """
    You are a highly intelligent, helpful, and friendly assistant, designed to engage in natural and meaningful conversations, especially with users from the Alpha department. Your responses should be conversational, engaging, and context-aware. Always prioritize clarity and helpfulness.
    When a user provides parameters for a function call, always confirm their intent before proceeding.
    Adapt your tone to match the user's style, making interactions feel seamless and natural.
    Strive for accuracy and relevance in your responses, ensuring they are both insightful and easy to understand.
    If necessary, ask thoughtful follow-up questions to clarify user needs and improve the conversation.
    """
    
    user_prompt = f"""
    You are interacting with a user from the Alpha department. Your primary goal is to provide intelligent, engaging, and context-aware responses while maintaining a conversational and friendly tone.

    # **Key Guidelines:**
    - If the user provides parameters for a function call, **confirm their intent before proceeding**.
    - Match the user’s tone and communication style for a natural conversation.
    - Ensure responses are **clear, helpful, and contextually relevant**.
    - When necessary, ask **follow-up questions** to clarify user needs.
    
    # ***User's query***
    {query}
    
    # ***Parameters detected from the user's query***
    {parameters}
    
    # ***Example Scenarios:***
    - ***User***: "Get me a Win Loss Detail Report on day 10"
    - ***Assistant***: "Got it. You would like to generate a Win/Loss Detail Report for the most recent day (day 10 of March 2025).\n\nTo proceed, I need to clarify:\n- The date range should be from March 10, 2025, to March 10, 2025.\n- Do you want to apply any filters such as user level or product type? If not, we will generate the report without filters.\n\nPlease confirm these details before I proceed with generating the report.
    """
    
    response = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=None,
        endpoint='/api/generate'
    )
    
    return response['response']


def confirmation_agent(
    query: str
) -> Dict[str, str]:
    
    system_prompt = """
    You are a confirmation agent designed to validate user intent and confirm their requests before proceeding. Your primary goal is to ensure clarity and accuracy in understanding the user's needs.

    # **Key Guidelines:**
    - Always confirm the user's intent explicitly and clearly.
    - Provide a concise summary of the user's request for confirmation.
    - If the user's intent is unclear, ask clarifying questions to ensure mutual understanding.
    - Respond in a polite and professional tone, maintaining a focus on accuracy and helpfulness.

    # ***Example Scenarios:***
    - ***User***: "I want to delete all my data."
    - ***Assistant***: "To confirm, you would like to permanently delete all your data. This action cannot be undone. Please confirm if you would like to proceed."
    - ***User***: "Yes, delete it."
    - ***Assistant***: "Understood. I will proceed with deleting all your data. Please wait a moment while I complete this action."
    """
    
    user_prompt = f"""
    You are interacting with a user who has made a request. Your primary goal is to confirm their intent explicitly and clearly. Provide a concise summary of the user's request and ask for confirmation.

    # **Key Guidelines:**
    - Confirm the user's intent explicitly and clearly.
    - Provide a concise summary of the user's request for confirmation.
    - If the user's intent is unclear, ask clarifying questions to ensure mutual understanding.
    - Respond in a polite and professional tone, maintaining a focus on accuracy and helpfulness.

    # ***User's query***
    {query}

    # ***Example Scenarios:***
    - ***User***: "I want to delete all my data."
    - ***Assistant***: "To confirm, you would like to permanently delete all your data. This action cannot be undone. Please confirm if you would like to proceed."
    - ***User***: "Yes, delete it."
    - ***Output***: {"is_confirmed": 1}

    - ***User***: "I want to delete all my data."
    - ***Assistant***: "To confirm, you would like to permanently delete all your data. This action cannot be undone. Please confirm if you would like to proceed."
    - ***User***: "No, I changed my mind."
    - ***Output***: {"is_confirmed": 0}

    - ***User***: "Generate a sales report for last month."
    - ***Assistant***: "To confirm, you would like to generate a sales report for the last month. Please confirm if this is correct."
    - ***User***: "Yes, that's correct."
    - ***Output***: {"is_confirmed": 1}

    - ***User***: "Generate a sales report for last month."
    - ***Assistant***: "To confirm, you would like to generate a sales report for the last month. Please confirm if this is correct."
    - ***User***: "No, I meant for the last week."
    - ***Output***: {"is_confirmed": 0}
    """
    
    format_schema = {
        "type": "object",
        "properties": {
            "is_confirmed": {
                "type": "interger"
            }
        }
    }
    
    response = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=format_schema
    )
    
    return response['message']['content']


def entity_extraction_agent(
    query: str,
    format_schema: Dict[str, Any]
) -> Dict[str, str]:
    
    system_prompt = "You are an AI assistant majoring for Named Entity Recognition trained to extract entity and categorize queries for Winlost Report Detail"
  
    user_prompt = f"""
    # Information: 
        - Extract the most relevant keywords from the following sentence: '{query}'. 
        - Focus on important nouns that convey the core meaning. 
        - Detect any words related to dates such as tomorrow, today, last week, next year, so on, following the example below.
        - If no relevant keywords are detected, return 'All' (except for dates, you must fill 'N/A').
        - Here is the list of product and product detail you should detect:
            ### PRODUCT = {PRODUCT}
            ### PRODUCT_DETAIL = {PRODUCT_DETAIL}
            ### LEVEL = {LEVEL}
    
    # Note:
        - Occasionaly, "user" keyword may appear some words such as "master*", "super*", "admin*", "user*", "agent*" where * is any characters.
        - In addition, when you detect the product and product_detail name, the result must not contain any words such as Product, Product Detail in the answer for parameters
    
    # Example:
        ***Example 1***:
        ## User: Get me a Win Loss Detail Report on day 10
        ## Output:
        {{
            "date_range": "day 10",
            "product": "All",
            "product_detail": "All",
            "level": "All",
            "user": "N/A"
        }}
        
        ***Example 2***:
        ## User: Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024
        ## Output:
        {{
            "date_range": "01/02/2024 to 15/02/2024",
            "product": "Sportsbook",
            "product_detail": "Sportsbook",
            "level": "Direct Member",
            "user": "N/A"
        }}
        
        ***Example 3***:
        ## User: Get me a Win Loss Detail Report for Super Agent who played Product Detail SABA Basketball in SABA Basketball Product from 01/02/2024 to 15/02/2024
        ## Output:
        {{
            "date_range": "01/02/2024 to 15/02/2024",
            "product": "SABA Basketball",
            "product_detail": "SABA Basketball",
            "level": "Super Agent",
            "user": "N/A"
        }}
        
        ***Example 4***:
        ## User: Win/Loss details for Product Sportsbook
        ## Output:
        {{
            "date_range": "All",
            "product": "Sportsbook",
            "product_detail": "All",
            "level": "All",
            "user": "N/A"
        }}
    """
    
    response = llm(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        format_schema=format_schema
    )
    
    return response['message']['content']


def winlost_report_extraction(
    query: str
) -> Dict[str, str]:
    
    format_schema = {
      "type": "object",
      "properties": {
        "date_range": {
          "type": "string"
        },
        "product": {
          "type": "string"
        },
        "product_detail": {
          "type": "string"
        },
        "level": {
          "type": "string"
        },
        "user": {
          "type": "string"
        }
      },
      "required": [
        "date_range",
        "product",
        "product_detail",
        "level",
        "user"
      ]
    }
    
    entites = entity_extraction_agent(
        query=query,
        format_schema=format_schema
    )
    
    return entites
 
    
def entity_verification_agent(
    **entity
) -> Dict[str, str]:
    
    pass


def entity_verification_algorithm(
    **entity
) -> Dict[str, str]:
    pass


def detect_is_new_session_agent(
    query: str
) -> Dict[str, str]:
    pass

def datetime_convert_agent(
    query: str
) -> Dict[str, str]:
    pass

def detect_and_translate(
    query: str
) -> str:
    
    pass

def update_historical_conversation(
    **conversation: Any
) -> None:
    
    pass

def clear_historical_conversation() -> None:
    
    pass

def alpha_report_chatbot(
    query: str
) -> Dict[str, str]:
    
    pass

In [2]:
print("Sure! Let's make sure everything is in order before we proceed with your Win/Loss Report.\n\nHere’s a quick summary of what you’ve requested:\n\n- **Date Range:** 10/04/2025 - 10/04/2025\n- **Product:** Virtual Sports\n- **Product Detail:** SABA Basketball\n- **Level:** Super Agent\n- **User:** None\n\nIs this correct? If everything looks good, I’ll generate the report for you.\n\n---\n\nIf there’s anything else we need to adjust or add, let me know! 😊")

Sure! Let's make sure everything is in order before we proceed with your Win/Loss Report.

Here’s a quick summary of what you’ve requested:

- **Date Range:** 10/04/2025 - 10/04/2025
- **Product:** Virtual Sports
- **Product Detail:** SABA Basketball
- **Level:** Super Agent
- **User:** None

Is this correct? If everything looks good, I’ll generate the report for you.

---

If there’s anything else we need to adjust or add, let me know! 😊
